In [7]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import onnx
import keras2onnx
from numpy import random
import onnxruntime
import time

In [3]:
input_shape = (450,)
model_name_onnx = "model.1.onnx"

In [27]:
:
    return keras.Sequential(
        [
            keras.Input(shape=input_shape, name="input"),
            layers.Conv2D(16, kernel_size=(5, 5), activation="relu"),
            layers.Conv2D(8, kernel_size=(3, 3), activation="relu"),
            layers.Flatten(),
            #layers.Dropout(0.5),
            layers.Dense(2, activation="relu"),
            layers.Dense(1, activation="sigmoid", name="output")
        ]
    )
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 11, 11, 16)        816       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 8)           1160      
_________________________________________________________________
flatten (Flatten)            (None, 648)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1298      
_________________________________________________________________
output (Dense)               (None, 1)                 3         
Total params: 3,277
Trainable params: 3,277
Non-trainable params: 0
_________________________________________________________________


In [28]:
onnx_model = keras2onnx.convert_keras(model, model.name)

onnx.save_model(onnx_model, model_name_onnx)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 13
The maximum opset needed by this model is only 11.


In [5]:
session = onnxruntime.InferenceSession(model_name_onnx)  

In [12]:
data = random.rand(100000,input_shape[0]).astype(np.single)
print(data)
print(type(data[0]))

[[0.73856115 0.7738114  0.8009448  ... 0.9994568  0.57522446 0.18425141]
 [0.95285565 0.33014905 0.8106778  ... 0.2565606  0.7295704  0.7801375 ]
 [0.28436592 0.06623707 0.7933404  ... 0.38961726 0.99945205 0.3169882 ]
 ...
 [0.2903146  0.6607235  0.22211581 ... 0.27009493 0.14419587 0.9608937 ]
 [0.44504938 0.22011058 0.7903991  ... 0.16310783 0.85489684 0.67605686]
 [0.9968452  0.53456026 0.20096542 ... 0.12787887 0.9883669  0.62394154]]
<class 'numpy.ndarray'>


In [13]:
print("starting inference")
start = time.time()
for i in range(100):
    prediction = session.run(None, {"input": data} )
duration = time.time() - start
print(f"inference ended in {duration} seconds")

starting inference


Fail: [ONNXRuntimeError] : 1 : FAIL : Non-zero status code returned while running Transpose node. Name:'PushTranspose_216064' Status Message: CUDA error cudaErrorInvalidConfiguration:invalid configuration argument